# 1.1.5.1: Descomposición de Valor Singular (SVD)

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Explicar** la SVD como la descomposición de **cualquier** matriz $A$ en $U \Sigma V^T$.
- **Interpretar** geométricamente cada componente: Rotación ($V^T$), Escalamiento ($\Sigma$), y Rotación ($U$).
- **Utilizar** la SVD para construir **aproximaciones de rango reducido** para la compresión de datos.
- **Resolver** problemas de mínimos cuadrados de forma robusta usando la **Pseudo-inversa**, derivada de la SVD.
- **Conectar** explícitamente la SVD con el **Análisis de Componentes Principales (PCA)**, entendiendo por qué es el método numéricamente preferido.

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import datasets # Para la imagen de ejemplo

def plot_svd_transformation(matrix):
    t = np.linspace(0, 2*np.pi, 100)
    circle = np.vstack((np.cos(t), np.sin(t)))
    U, s, Vt = np.linalg.svd(matrix)
    S = np.diag(s)
    # Asegurarse que las matrices tengan las dimensiones correctas para 2D
    if U.shape[0] > 2:
        U = U[:, :2]
    if Vt.shape[0] > 2:
        Vt = Vt[:2, :]
    if S.shape[0] > 2:
        S = S[:2, :2]
        
    step1 = Vt @ circle
    step2 = S @ step1
    step3 = U @ step2
    
    fig, axs = plt.subplots(1, 4, figsize=(22, 5.5))
    titles = ['1. Original', '2. Rotación por $V^T$', '3. Escalamiento por $\Sigma$', '4. Rotación por U (Final)']
    data = [circle, step1, step2, step3]
    
    for i, ax in enumerate(axs):
        ax.plot(data[i][0, :], data[i][1, :], lw=3)
        ax.set_title(titles[i], fontsize=14)
        ax.axis('equal'); ax.grid(True)
        ax.axhline(0, color='black', lw=0.5); ax.axvline(0, color='black', lw=0.5)
        
    plt.suptitle('Descomposición Geométrica de una Transformación vía SVD', fontsize=18, y=1.02)
    plt.show()

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

La SVD es la "navaja suiza" del álgebra lineal. Para explorarla, necesitamos datasets que nos permitan ver sus superpoderes: compresión, solución de sistemas y su conexión con PCA. Usaremos imágenes, datos de estudiantes y matrices con propiedades especiales.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_edge_cases import create_edge_cases
from src.data_generation.create_special_matrices import create_special_matrices

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets y Matrices para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor (Aplicación a PCA)
# La conexión entre SVD y PCA es uno de los resultados más importantes. Usaremos
# nuestros datos de estudiantes para demostrar que la SVD de la matriz de datos nos da
# directamente los componentes principales, de una forma más estable que la eigendescomposición.
datos_estudiantes = create_student_performance_data(rng, simplified=True, n_samples=200)

# 💡 CONTEXTO PEDAGÓGICO: Compresión de Datos
# Una imagen es solo una matriz de valores de píxeles. Esto la convierte en el ejemplo
# perfecto para visualizar la aproximación de rango reducido, la base de la compresión.
imagen_mapache = datasets.face(gray=True)

# 💡 CONTEXTO PEDAGÓGICO: Mínimos Cuadrados Robusto
# Usaremos un dataset con multicolinealidad para demostrar cómo la pseudo-inversa,
# calculada con SVD, puede resolver problemas de mínimos cuadrados donde las Ecuaciones Normales fallan.
datos_multicolineales = create_edge_cases(rng, case_type='multicollinear', n_samples=50)

print("Datasets y matrices generados y listos para usar.")

## 1. La Gran Intuición: Toda Matriz es Rotación, Escalamiento y Rotación

La Descomposición de Valor Singular (SVD) es posiblemente el teorema más importante del álgebra lineal aplicada. Afirma que **cualquier** matriz $A$ de $m \times n$ (incluso no cuadrada) puede ser factorizada en el producto de tres matrices con interpretaciones geométricas muy claras:
$$ A = U \Sigma V^T $$

- **$V^T$ (Primera Rotación):** Una matriz **ortogonal** ($n \times n$) que rota el espacio de entrada sin cambiar longitudes ni ángulos. Alinea los ejes del espacio de entrada con los ejes principales de la transformación.
- **$\Sigma$ (Escalamiento):** Una matriz **diagonal** ($m \times n$) que estira o encoge el espacio a lo largo de los ejes rotados. Sus elementos diagonales $\sigma_i$ son los **valores singulares** de A, y están ordenados de mayor a menor ($\sigma_1 \ge \sigma_2 \ge \dots \ge 0$).
- **$U$ (Segunda Rotación):** Otra matriz **ortogonal** ($m \times m$) que rota el espacio de salida resultante a su orientación final.

La SVD nos da un desglose completo de la "receta" de cualquier transformación lineal: una rotación, un escalamiento puro a lo largo de ejes perpendiculares, y una rotación final.

### Ejemplo Demostrativo 1: Visualización Geométrica de la SVD

In [ ]:
# 1. DATOS: Una matriz de transformación 2x2 simple.
A = np.array([[3, 0], [2, 2]])

# 2. APLICACIÓN Y VISUALIZACIÓN
# La función de ayuda `plot_svd_transformation` calcula la SVD y aplica cada paso
# a un círculo unitario para que podamos ver la descomposición de la acción.
plot_svd_transformation(A)

## 2. Aplicación 1: Aproximación de Rango Reducido (Compresión)

Los valores singulares en $\Sigma$ están ordenados por "importancia". El primer valor singular, $\sigma_1$, captura la dirección de mayor acción de la matriz. Al quedarnos solo con los `k` valores singulares más grandes, podemos reconstruir una matriz $A_k = U_k \Sigma_k V_k^T$ que es la **mejor aproximación posible de rango `k`** a la matriz original.

Esto es la base matemática de la **compresión con pérdida** (como en JPEG), el **filtrado de ruido** en datos, y los **sistemas de recomendación** (donde se aproxima la matriz gigante de usuario-ítem).

### Ejemplo Demostrativo 2: Compresión de Imagen

In [ ]:
# 1. DATOS: Una imagen es solo una matriz de valores de píxeles.
img = imagen_mapache

# 2. APLICACIÓN: Calculamos la SVD de la matriz de la imagen.
U, s, Vt = np.linalg.svd(img)
Sigma = np.diag(s) # Construimos la matriz diagonal Sigma

print(f"Dimensiones originales: U:{U.shape}, s:{s.shape}, Vt:{Vt.shape}")

# 3. RECONSTRUCCIÓN Y VISUALIZACIÓN: Reconstruimos la imagen usando diferentes números de valores singulares (k).
fig, axs = plt.subplots(1, 5, figsize=(25, 5))
ranks = [5, 20, 50, 100, len(s)]

for i, k in enumerate(ranks):
    # Tomamos las primeras k columnas de U, k valores singulares, y k filas de Vt
    A_k = U[:, :k] @ Sigma[:k, :k] @ Vt[:k, :]
    
    # Calculamos el ratio de compresión (aproximado)
    original_size = img.shape[0] * img.shape[1]
    compressed_size = U[:, :k].size + s[:k].size + Vt[:k, :].size
    ratio = compressed_size / original_size * 100
    
    axs[i].imshow(A_k, cmap='gray')
    title = f'Original' if k == len(s) else f'Rango k = {k}'
    axs[i].set_title(f'{title}\n({ratio:.1f}% del tamaño)')
    axs[i].axis('off')

plt.suptitle('Compresión de Imagen usando SVD de Rango Reducido', fontsize=18, y=1.02); plt.show()

## 3. Aplicación 2: La Pseudoinversa y Mínimos Cuadrados

Cuando resolvimos $X\vec{\beta}=\vec{y}$ con las Ecuaciones Normales, necesitábamos que $X^T X$ fuera invertible. Pero, ¿qué pasa si hay multicolinealidad perfecta y $X^T X$ es singular? La SVD nos da una solución más general y numéricamente estable: la **pseudoinversa de Moore-Penrose**.

La pseudoinversa de $A = U \Sigma V^T$ se calcula como $ A^+ = V \Sigma^+ U^T $, donde $\Sigma^+$ se obtiene de $\Sigma$ tomando el recíproco ($1/\sigma_i$) de los valores singulares **no nulos**.

La solución de mínimos cuadrados de norma mínima para $A\vec{x}=\vec{b}$ es simplemente:
$$ \hat{x} = A^+ \vec{b} $$
Este método es el más robusto para resolver regresiones lineales, y es lo que funciones como `np.linalg.lstsq` usan internamente.

### Ejemplo Demostrativo 3: SVD para Regresión Robusta

In [ ]:
# 1. DATOS: Usamos el dataset con multicolinealidad perfecta.
X_raw = datos_multicolineales[['x1', 'x2', 'x3']].values
X = np.c_[np.ones(X_raw.shape[0]), X_raw] # Añadimos intercepto
y = datos_multicolineales['y'].values

# 2. APLICACIÓN: Resolvemos usando la pseudoinversa.
X_plus = np.linalg.pinv(X) # Calculamos la pseudoinversa
beta_hat = X_plus @ y

# 3. INTERPRETACIÓN
print("Matriz X (con multicolinealidad perfecta):")
print(X[:5])
print(f"\nNúmero de Condición de XᵀX: {np.linalg.cond(X.T @ X):.2e} (Extremadamente alto)")
print("\nLas Ecuaciones Normales fallarían aquí.")
print(f"\nSolución de Mínimos Cuadrados vía Pseudoinversa (SVD):")
print(f"β_hat = {np.round(beta_hat, 4)}")
print("\nLa SVD encuentra una solución estable y de norma mínima incluso en casos de dependencia lineal perfecta.")

## 4. Conexión Final: SVD y PCA

La conexión entre SVD y PCA es profunda y elegante. Mientras que PCA se define a través de la eigendescomposición de la matriz de covarianza, resulta que la SVD de la matriz de datos (centrada) nos da los mismos resultados de una forma numéricamente más estable.

- Las **columnas de $V$** (los vectores singulares derechos) son los **Componentes Principales** (los eigenvectores de la matriz de covarianza).
- Los **valores singulares ($s$) al cuadrado** son proporcionales a los **eigenvalores** de la matriz de covarianza (la varianza explicada por cada componente).

---
## 5. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión.

### === EJERCICIO GUIADO 1: Descomponiendo una Matriz ===

In [ ]:
# DATOS
A = np.array([[1, 2, 3], [4, 5, 6]])

# TODO: Calcula la SVD de A.
# PISTA: Usa np.linalg.svd().
U, s, Vt = # COMPLETAR

# VERIFICACIÓN
assert U.shape == (2, 2)
assert s.shape == (2,)
assert Vt.shape == (3, 3)
print("✅ ¡SVD calculada correctamente!")
print(f"Forma de U: {U.shape}")
print(f"Valores singulares s: {s}")
print(f"Forma de Vᵀ: {Vt.shape}")

### === EJERCICIO GUIADO 2: Reconstruyendo desde SVD ===

In [ ]:
# DATOS: Los componentes U, s, Vt del ejercicio anterior.
A = np.array([[1, 2, 3], [4, 5, 6]])
U, s, Vt = np.linalg.svd(A)

# Para reconstruir, Sigma (Σ) debe tener la misma forma que A.
# TODO 1: Crea una matriz de ceros con la forma de A.
Sigma = # COMPLETAR

# TODO 2: Rellena la diagonal de Sigma con los valores singulares 's'.
# PISTA: Sigma[:A.shape[1], :A.shape[1]] = np.diag(s)
# COMPLETAR

# TODO 3: Reconstruye A multiplicando U @ Sigma @ Vt.
A_reconstruida = # COMPLETAR

# VERIFICACIÓN
assert np.allclose(A, A_reconstruida)
print("✅ ¡Matriz reconstruida con éxito!")
print(np.round(A_reconstruida, 2))

### === EJERCICIO GUIADO 3: Aproximación de Rango 1 ===

In [ ]:
# DATOS: U, s, Vt de la matriz A.
A = np.array([[1, 2, 3], [4, 5, 6]])
U, s, Vt = np.linalg.svd(A)
k = 1 # Rango de la aproximación

# TODO 1: Selecciona las primeras 'k' columnas de U.
Uk = # COMPLETAR

# TODO 2: Selecciona los primeros 'k' valores singulares y ponlos en una matriz diagonal.
Sk = # COMPLETAR

# TODO 3: Selecciona las primeras 'k' filas de Vt.
Vtk = # COMPLETAR

# TODO 4: Calcula la aproximación de rango k.
A_k = # COMPLETAR

assert A_k.shape == A.shape
print(f"✅ Aproximación de Rango {k} calculada:")
print(np.round(A_k, 2))

### === EJERCICIO GUIADO 4: Resolver OLS con Pseudoinversa ===

In [ ]:
# DATOS: Matriz de diseño X y vector y del Hilo Conductor.
X_feature = datos_estudiantes[['horas_estudio']].values
X = np.c_[np.ones(X_feature.shape[0]), X_feature]
y = datos_estudiantes['calificacion_examen'].values

# TODO 1: Calcula la pseudoinversa de X.
# PISTA: Usa np.linalg.pinv().
X_plus = # COMPLETAR

# TODO 2: Calcula la solución de mínimos cuadrados beta_hat.
beta_hat = # COMPLETAR

# VERIFICACIÓN
beta_lstsq, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
assert np.allclose(beta_hat, beta_lstsq)
print("✅ ¡OLS resuelto con pseudoinversa!")
print(f"β_hat = {np.round(beta_hat, 2)}")

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Cálculo y Aproximación

**A1 (🟢 Fácil):** Calcula la SVD de $A = \begin{pmatrix} 3 & 0 \\ 0 & -2 \end{pmatrix}$. ¿Qué son U, s y Vt en este caso simple?

**A2 (🟢 Fácil):** Calcula la SVD de la matriz de rotación $R = \begin{pmatrix} 0 & -1 \\ 1 & 0 \end{pmatrix}$. ¿Qué valores singulares obtienes? ¿Tiene sentido?

**A3 (🟡 Medio):** Dada la matriz $M = \begin{pmatrix} 1 & 1 & 0 \\ 0 & 1 & 1 \end{pmatrix}$, calcula su SVD y su mejor aproximación de rango 1.

**A4 (🟡 Medio):** Genera una matriz aleatoria de 100x50. Calcula su SVD y luego reconstrúyela usando solo los 10 valores singulares más grandes.

**A5 (🔴 Reto):** La norma de Frobenius de una matriz es $\|A\|_F = \sqrt{\sum_{i,j} A_{ij}^2}$. También es igual a la raíz cuadrada de la suma de los cuadrados de sus valores singulares. Genera una matriz 5x3 aleatoria y verifica esta propiedad.

### Parte B: Aplicaciones (Pseudoinversa y PCA)

**B1 (🟢 Fácil):** Usa `np.linalg.pinv` para calcular la pseudoinversa de $A = \begin{pmatrix} 1 \\ 2 \end{pmatrix}$ (un vector columna).

**B2 (🟡 Medio):** Usa `np.linalg.pinv` para calcular la pseudoinversa de la matriz singular $S = \begin{pmatrix} 1 & 2 \\ 2 & 4 \end{pmatrix}$.

**B3 (🟡 Medio):** Resuelve el problema de regresión simple para `datos_estudiantes` usando la pseudoinversa y verifica que los coeficientes coinciden con los de `lstsq`.

**B4 (🔴 Reto):** Resuelve la regresión múltiple para el dataset `datos_multicolineales` usando la pseudoinversa. Compara tus coeficientes $\beta$ con los que obtendrías de `lstsq`.

**B5 (🔴 Reto):** Toma los datos de estudiantes centrados del Ejemplo 3. Proyecta los datos sobre el primer componente principal (la primera columna de V, o `Vt.T[:, 0]`). El resultado es la "coordenada" de cada estudiante a lo largo del eje más importante. Haz un histograma de estas coordenadas.

---

## ✅ Mini-Quiz de Autoevaluación

*Responde estas preguntas para verificar tu comprensión.*

1. ¿Cuáles son las tres matrices que componen la SVD de una matriz A y qué representa geométricamente cada una?
2. ¿Para qué tipo de problemas de regresión es la pseudoinversa (calculada vía SVD) particularmente útil?
3. ¿Qué componentes de la SVD de una matriz de datos (centrada) corresponden a los Componentes Principales de PCA?
4. Verdadero o Falso: La SVD solo se puede aplicar a matrices cuadradas.

## 🚀 Próximos Pasos (Cierre del Área 1.1)

**¡Felicidades, has llegado a la cima del Álgebra Lineal aplicada!** Con la SVD, tienes la herramienta más poderosa y robusta para analizar y manipular matrices. Has conectado todos los puntos, desde la definición de un vector hasta la implementación de PCA usando SVD, demostrando cómo la teoría abstracta potencia las aplicaciones más importantes de la ciencia de datos.

- **Siguiente Parada:** **Área 1.2: Cálculo**. Cambiaremos de marcha para explorar la matemática del cambio. Descubriremos cómo las derivadas nos permiten encontrar la "pendiente" en espacios de alta dimensión, una idea fundamental para entrenar y optimizar casi todos los modelos de Machine Learning mediante algoritmos como el Descenso de Gradiente.